In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


# Scripts for generating events and running environment

In [2]:
from datetime import datetime, timedelta
import os
import pytz

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

from sustaingym.data.load_moer import load_monthly_moer
from sustaingym.envs.evcharging import EVChargingEnv, RealTraceGenerator, GMMsTraceGenerator

test_ranges = (
    ('2019-05-01', '2019-08-31'),
    ('2019-09-01', '2019-12-31'),
    ('2020-02-01', '2020-05-31'),
    ('2021-05-01', '2021-08-31'),
)


## Real trace generator

In [3]:
for site in ['caltech', 'jpl']:
    for test_range in test_ranges:
        print('testing: ', site, test_range)
        rtg = RealTraceGenerator(site, test_range)
        for _ in range(123):  # 4 months -> 123 days maximum
            _, _, num_plug_events = rtg.get_event_queue()
        print(rtg)

testing:  caltech ('2019-05-01', '2019-08-31')
RealTracesGenerator from the Caltech site from 2019-05-01 to 2019-08-31. Current day 2019-08-31. 
testing:  caltech ('2019-09-01', '2019-12-31')
RealTracesGenerator from the Caltech site from 2019-09-01 to 2019-12-31. Current day 2019-09-01. 
testing:  caltech ('2020-02-01', '2020-05-31')
RealTracesGenerator from the Caltech site from 2020-02-01 to 2020-05-31. Current day 2020-02-02. 
testing:  caltech ('2021-05-01', '2021-08-31')
RealTracesGenerator from the Caltech site from 2021-05-01 to 2021-08-31. Current day 2021-08-31. 
testing:  jpl ('2019-05-01', '2019-08-31')
RealTracesGenerator from the Jpl site from 2019-05-01 to 2019-08-31. Current day 2019-08-31. 
testing:  jpl ('2019-09-01', '2019-12-31')
RealTracesGenerator from the Jpl site from 2019-09-01 to 2019-12-31. Current day 2019-09-01. 
testing:  jpl ('2020-02-01', '2020-05-31')
RealTracesGenerator from the Jpl site from 2020-02-01 to 2020-05-31. Current day 2020-02-02. 
testing: 

## GMMs trace generator

In [4]:
for site in ['caltech', 'jpl']:
    for test_range in test_ranges:
        print('testing: ', site, test_range)
        gmmg = GMMsTraceGenerator(site, test_range)
        for _ in range(123):  # 4 months -> 123 days maximum
            _, _, num_plug_events = gmmg.get_event_queue()
        print(gmmg)

testing:  caltech ('2019-05-01', '2019-08-31')
GMMsTracesGenerator from the Caltech site from 2019-05-01 to 2019-08-31. Sampler is GMM with 30 components. 
testing:  caltech ('2019-09-01', '2019-12-31')
GMMsTracesGenerator from the Caltech site from 2019-09-01 to 2019-12-31. Sampler is GMM with 30 components. 
testing:  caltech ('2020-02-01', '2020-05-31')
GMMsTracesGenerator from the Caltech site from 2020-02-01 to 2020-05-31. Sampler is GMM with 30 components. 
testing:  caltech ('2021-05-01', '2021-08-31')
GMMsTracesGenerator from the Caltech site from 2021-05-01 to 2021-08-31. Sampler is GMM with 30 components. 
testing:  jpl ('2019-05-01', '2019-08-31')
GMMsTracesGenerator from the Jpl site from 2019-05-01 to 2019-08-31. Sampler is GMM with 30 components. 
testing:  jpl ('2019-09-01', '2019-12-31')
GMMsTracesGenerator from the Jpl site from 2019-09-01 to 2019-12-31. Sampler is GMM with 30 components. 
testing:  jpl ('2020-02-01', '2020-05-31')
GMMsTracesGenerator from the Jpl site

## Rewards from policies

In [3]:
policies = ['random', 'full', 'none', 'selective_full']

for policy in policies:
    rewards = []
    reward_comps = {'profit': 0, 'carbon_cost': 0, 'excess_charge': 0}
    gmmg = GMMsTraceGenerator('caltech', test_ranges[0])
    env = EVChargingEnv(gmmg, moer_forecast_steps=36)

    num_episodes = 5
    for _ in range(num_episodes):
        obs = env.reset()
        done = False
        tot_reward = 0
        while not done:
            if policy == 'random':
                action = np.random.randint(0, 5, size=(54,))
            elif policy == 'full':
                action = np.full((54,), 4)
            elif policy == 'none':
                action = np.zeros((54,))
            else:
                action = np.where(obs['demands'] > 0, 4, 0)
            obs, reward, done, info = env.step(action)

            tot_reward += reward
            for reward_comp in info['reward']:
                reward_comps[reward_comp] += info['reward'][reward_comp]
        rewards.append(tot_reward)
    print(f'{policy} rewards: ', rewards)  # rewards per num_episodes
    print('reward components:', reward_comps)  # total reward contribution over num_episodes


random rewards:  [-14.449023650967707, -14.110085873847723, -34.360503854055864, -31.797721305261057, -42.031788122321146]
reward components: {'profit': 30.39614692914331, 'carbon_cost': 154.15423033569135, 'excess_charge': 12.991039399905606}
full rewards:  [-101.2611915271211, -50.97029626573386, -18.31240343947611, -78.33175308205146, -38.51051357212903]
reward components: {'profit': 60.18811874727833, 'carbon_cost': 275.18351125567017, 'excess_charge': 72.39076537812032}
none rewards:  [0.0, 0.0, 0.0, 0.0, 0.0]
reward components: {'profit': 0.0, 'carbon_cost': 0.0, 'excess_charge': 0.0}
selective_full rewards:  [12.374412825986386, 13.44284552680034, 16.01621332748467, 12.733164961240538, 13.147108333150314]
reward components: {'profit': 77.90593530354735, 'carbon_cost': 9.695512607993402, 'excess_charge': 0.4966777208917618}


## Check seed setting is done correctly

In [3]:
print('RealTraceGenerator')
for sequential in [False, True]:
    for random_seed in [None, 11]:
        print(f'sequential {sequential}, random_seed {random_seed}')
        for _ in range(3):
            gen = RealTraceGenerator('caltech', test_ranges[1], sequential=sequential, random_seed=random_seed)

            num_events = []
            for _ in range(7):
                _, _, num_event = gen.get_event_queue()
                num_events.append(num_event)
            print(num_events)

print('GMMsTraceGenerator')
for random_seed in [None, 11]:
    print(f'random_seed {random_seed}')
    for _ in range(3):
        gen = GMMsTraceGenerator('caltech', test_ranges[1], random_seed=random_seed)
        num_events = []
        for _ in range(7):
            _, _, num_event = gen.get_event_queue()
            num_events.append(num_event)
        print(num_events)

RealTraceGenerator
sequential False, random_seed None
[29, 10, 36, 26, 23, 27, 31]
[34, 32, 35, 34, 28, 28, 4]
[31, 31, 9, 4, 1, 6, 30]
sequential False, random_seed 11
[32, 6, 20, 33, 23, 33, 30]
[32, 6, 20, 33, 23, 33, 30]
[32, 6, 20, 33, 23, 33, 30]
sequential True, random_seed None
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
sequential True, random_seed 11
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
GMMsTraceGenerator
random_seed None
[36, 9, 6, 29, 7, 30, 38]
[30, 31, 31, 35, 4, 11, 33]
[7, 33, 1, 8, 6, 31, 7]
random_seed 11
[35, 10, 39, 38, 30, 19, 9]
[35, 10, 39, 38, 30, 19, 9]
[35, 10, 39, 38, 30, 19, 9]
